<a href="https://colab.research.google.com/github/veerashayyagari/llms-in-prod/blob/main/Miami_Hotels_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"

locale.getpreferredencoding = getpreferredencoding

In [2]:
!pip install -U openai gradio sentence_transformers accelerate spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 105.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.3/140.3 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import pandas as pd
url = 'https://raw.githubusercontent.com/hamzafarooq/maven-mlsystem-design-cohort-1/main/data/miami_hotels.csv'
df = pd.read_csv(url)
df.shape

(2511, 27)

In [4]:
df.head()

,id,type,name,image,awards,rankingPosition,priceLevel,priceRange,category,rating,...,prices,latitude,longitude,webUrl,website,rankingString,rankingDenominator,numberOfReviews,review,title
0,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,[],25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Hands down my absolute favorite hotel in South...,My favorite hotel in Miami--Petar is the best!
1,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,[],25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,There are not enough words to explain the sump...,Exquisite Stay
2,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,[],25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I went for the Burlesque show which was absolu...,Burlesqu Show
3,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,[],25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Dana at Gitano made the night magical! He make...,Dana at Gitano is a rockstar!
4,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,[],25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I would like to give our very very heartfelt t...,Perfect Place


In [5]:
df['priceLevel'].value_counts()

$$$     1151
$$$$    1013
$$       300
$         47
Name: priceLevel, dtype: int64

In [6]:
def parse_price_level(level):
  s = 'relative price : '
  if level == '$':
    s += 'inexpensive'
  elif level == '$$':
    s += 'moderately expensive'
  elif level == '$$$':
    s += 'expensive'
  else:
    s += 'very expensive'
  return s

df['formatted_pricelevel'] = df['priceLevel'].apply(lambda x: parse_price_level(x))
df.head()

,id,type,name,image,awards,rankingPosition,priceLevel,priceRange,category,rating,...,latitude,longitude,webUrl,website,rankingString,rankingDenominator,numberOfReviews,review,title,formatted_pricelevel
0,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Hands down my absolute favorite hotel in South...,My favorite hotel in Miami--Petar is the best!,relative price : very expensive
1,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,There are not enough words to explain the sump...,Exquisite Stay,relative price : very expensive
2,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I went for the Burlesque show which was absolu...,Burlesqu Show,relative price : very expensive
3,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Dana at Gitano made the night magical! He make...,Dana at Gitano is a rockstar!,relative price : very expensive
4,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I would like to give our very very heartfelt t...,Perfect Place,relative price : very expensive


In [7]:
import re

def extract_min_max_price(price_range_str):
    # First, let's check if the value is a range
    range_match = re.search(r'\$(\d+[\d,]*)\s*-\s*\$(\d+[\d,]*)', price_range_str)

    if range_match:
        # Extract min and max prices for the range format (ex. $700 - $1000)
        min_price = float(range_match.group(1).replace(',', ''))
        max_price = float(range_match.group(2).replace(',', ''))
        return min_price, max_price
    else:
        # Now let's check for a single value format (ex. $1000)
        single_match = re.search(r'\$(\d+[\d,]*)', price_range_str)
        if single_match:
            single_price = float(single_match.group(1).replace(',', ''))
            return single_price, single_price  # Use the single price as both min and max

        # Return NaN values if no regex patterns match
        return (float('nan'), float('nan'))

df['min_price'], df['max_price'] = zip(*df['priceRange'].apply(extract_min_max_price))
df.head()

,id,type,name,image,awards,rankingPosition,priceLevel,priceRange,category,rating,...,webUrl,website,rankingString,rankingDenominator,numberOfReviews,review,title,formatted_pricelevel,min_price,max_price
0,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Hands down my absolute favorite hotel in South...,My favorite hotel in Miami--Petar is the best!,relative price : very expensive,729.0,1426.0
1,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,There are not enough words to explain the sump...,Exquisite Stay,relative price : very expensive,729.0,1426.0
2,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I went for the Burlesque show which was absolu...,Burlesqu Show,relative price : very expensive,729.0,1426.0
3,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Dana at Gitano made the night magical! He make...,Dana at Gitano is a rockstar!,relative price : very expensive,729.0,1426.0
4,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I would like to give our very very heartfelt t...,Perfect Place,relative price : very expensive,729.0,1426.0


In [8]:
import spacy

# Load spaCy's English model
nlp = spacy.load("en_core_web_sm")

def preprocess_text(text):
    # Process the review using spaCy
    doc = nlp(text)

    # Extract tokens that are not stopwords, punctuation, or whitespace, then lemmatize them
    tokens = [token.lemma_ for token in doc if not (token.is_stop or token.is_punct or token.is_space)]

    # Join tokens back into a single string and return
    return ' '.join(tokens)

In [9]:
columns_to_clean = ['title', 'review']
df = df.assign(**{f'cleaned_{col}': df[col].apply(preprocess_text) for col in columns_to_clean})

In [10]:
def add_ner_tags(text):
    # Process the text using spaCy NER
    doc = nlp(text)

    # Extract the recognized entities as a list of tags
    tags = [ent.text for ent in doc.ents]

    # Append the tags to the original text
    return ",".join(tags)

df['ner_review'] = df['review'].apply(add_ner_tags)
df['ner_review'][2]

'Burlesque,Patty,one,Tatiana,Patty'

In [11]:
df["text"] = ("title: "+df.cleaned_title.str.strip()+
              "; content: " + df.cleaned_review.str.strip()+
              "; rooms: " + df.numberOfRooms.astype(str)+
              "; ranked: " + df.rankingString.str.strip()+
              "; "+df.formatted_pricelevel+
              "; minimum price per person: "+
              df.min_price.astype(str)+
              "; maximum price per person: "
              +df.max_price.astype(str))
df['text'][2]

'title: Burlesqu; content: go Burlesque absolutely amazing service tier bartender Patty drink great good whiskey sour server Tatiana awesome sweet attentive highly recommend chance ask Patty bar; rooms: 179; ranked: #5 of 235 hotels in Miami Beach; relative price : very expensive; minimum price per person: 729.0; maximum price per person: 1426.0'

In [12]:
df['ner_text'] = df['text'] + "; tags: "+df['ner_review']

In [17]:
sdf = df[['text', 'ner_text', 'name', 'image', 'phone', 'address', 'email', 'priceLevel', 'prices', 'rating', 'latitude', 'longitude', 'website', 'rankingString', 'title', 'review']]

In [ ]:
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import torch

mpnet_embedder = SentenceTransformer('all-mpnet-base-v2')
bge_embedder = SentenceTransformer('BAAI/bge-large-en')

if not torch.cuda.is_available():
    print("Warning: No GPU found. Please add GPU to your notebook")
else:
  print("GPU Found!")
  mpnet_embedder =  mpnet_embedder.to('cuda')
  bge_embedder = bge_embedder.to('cuda')

In [18]:
sdf["mpnet_text_embedding"] = sdf.text.apply(mpnet_embedder.encode)
sdf["mpnet_text_with_ner_embedding"] = sdf.ner_text.apply(mpnet_embedder.encode)

<ipython-input-18-26b633c3c82e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdf["mpnet_text_embedding"] = sdf.text.apply(mpnet_embedder.encode)
<ipython-input-18-26b633c3c82e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdf["mpnet_text_with_ner_embedding"] = sdf.ner_text.apply(mpnet_embedder.encode)


In [20]:
sdf["bge_text_embedding"] = sdf.text.apply(lambda x: bge_embedder.encode(x, normalize_embeddings=True))
sdf["bge_text_with_ner_embedding"] = sdf.ner_text.apply(lambda x: bge_embedder.encode(x, normalize_embeddings=True))

<ipython-input-20-40a729a6ac34>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdf["bge_text_embedding"] = sdf.text.apply(lambda x: bge_embedder.encode(x, normalize_embeddings=True))
<ipython-input-20-40a729a6ac34>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdf["bge_text_with_ner_embedding"] = sdf.ner_text.apply(lambda x: bge_embedder.encode(x, normalize_embeddings=True))


In [21]:
sdf.to_pickle('pickle/embeddings.pkl')